<img src="https://lh3.googleusercontent.com/pw/AM-JKLVhTn_UySwMdfMwXvoq8l3VN7IkrY9cwtH2YJVMxAlMznUBWC9IpFtgPRIyfAXru4oykkYD-1WjWi0Ao5XgkB9JICvzDBcfn0L_5X2_KOOppsURK5DfSifCC-s7Vx5oQrBUn_BNWn_hfAPdhlVbKQGE=w1097-h235-no?authuser=0" alt="2021年度ゲノム情報解析入門" height="100px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

# 大規模データ解析入門 - Sliding Window解析 -

　前回から、**MutMap解析**というゲノムデータ解析をおこなっています。ゲノム全域にわたって、突然変異塩基の割合（**SNP-index**）を調べて、突然変異形質を引き起こす遺伝子領域を特定する解析です。前回は、SNP-indexの算出と散布図への描画をおこない、SNP-indexが高いSNP座を図示しました。

　今回、**Sliding Window解析** をおこない、SNP-indexが平均的に高くなるゲノムポジション（原因遺伝子座）を探しましょう。


### MutMap解析の概要

　MutMap解析は、次のような手順で解析を行います。

1. オリジナル系統に突然変異を誘発させて、突然変異系統を作成します。
  - 突然変異系統のゲノム上に多数の変異が入り、そのうちの一つが形質を変化させる突然変異です。
1. 突然変異系統とオリジナル系統を交配して、交配後第二世代（F2集団）を作成します。
  - F2集団を作ることで、染色体がシャッフルした個体を多数得ます。
  - ゲノム上に誘発された各突然変異は、F2の各個体に約50%の確率で伝えられます。
  - 突然変異形質をもつF2個体は、ゲノム中のある突然変異を共通にもっているはずです。
1. 突然変異形質を持つ個体のDNAを集め（以下バルクDNA）、次世代シーケンシングをおこないます。オリジナル系統も同時にシーケンシングします。
1. オリジナル系統とバルクDNAシーケンスの両者を比較して、次の情報が得られます。
  - 突然変異が誘発された位置
  - オリジナル系統の塩基（リファレンス塩基）と突然変異の塩基の種類
  - バルクDNAシーケンス中のリファレンス塩基と突然変異塩基の個数
1. ゲノム全域にわたり、バルクDNAシーケンス中の突然変異塩基の割合（**SNP-index**）を調べて、突然変異塩基のみが見つかる遺伝子領域（原因遺伝子領域）を特定します。

<img src="https://lh3.googleusercontent.com/pw/ACtC-3e5jUBTKSMRm9RM3cY2CsuHvlm0elGRVN6wdLgJ5qtOwUgknoazlf6Q2OxEhQHbGLj2eIBaxg5qa5He5vzldR2eW85M0eeZ2CjEVYoPTtbwJ7nJRsNRQEnBYYMxUaXyNyTESHezfQNYuIdhKMvDQttp=w720-h405-no?authuser=0" alt="mutmap_overview" height="400px">

<img src="https://lh3.googleusercontent.com/pw/ACtC-3fRw0qGyT5XIfYvTTqTHohy-mcueji1tWn-Br1UpDx0Q1pziUsZ_STGlke_qWa6Htb-1nEXPD2cjhhqpVoWbx8SyDBF6_YuHqekNWi36Unkl0IucozdOMl6LtQVp1IWHCRKiunyyOi1-wGlPUm5lEfr=w779-h335-no?authuser=0" alt="mutmap_overview_graph" height="200px">

　詳しく知りたい方は、下記の論文をご覧ください。

> Abe, A., Kosugi, S., Yoshida, K., Natsume, S., Takagi, H., Kanzaki, H., Matsumura, H., Yoshida, K., Mitsuoka, C., Tamiru, M., Innan, H., Cano, L., Kamoun, S., Terauchi, R. (2012). [Genome sequencing reveals agronomically important loci in rice using MutMap.](https://www.nature.com/articles/nbt.2095)  Nature biotechnology, 30(2), 174.


### 前回の実習「大規模データ解析入門 - MutMap解析 -」でおこなったこと

　このテキストでは、イネの葉が「淡緑色(pale green)」になる遺伝子の位置を突き止めようとしています。

<img src="https://lh3.googleusercontent.com/pw/ACtC-3c1W_Ul3wYElrGq35UuP6Otsy9zce5kpT1BXDxpWXug3fWOC5BtOT_fmyok5uakfwbtlGzFlYjH-gopl8YNNVa62uUzimtc4_OdNzcSwb1kVbqW7hcST2MsgMs2Qe14E9qIWH58gglNIgxL0WurxOZ-=w395-h216-no?authuser=0" alt="mutmap04" height="200px">

　前回、MutMap解析の一部、SNP-indexの算出と描画をおこないました。

- SNP-indexの算出
- 散布図描画
  - 全SNP-indexの描画
  - 高SNP-indexの描画


<img src="https://lh3.googleusercontent.com/pw/ACtC-3ehpu7szvJCN7Pw1vz9pGzeNiTQ6LXN-uj6ZIP-_5BoAmtwUnauQLrQlKxU5RMhiUZC0HgK0nzb7WkX7mhTgzb1sSzJbIP17aTujTsdBSu7oERctOYBMgB0w0bWNvjNfSMR9akDBWtLl9UiGBj8TaUD=w1456-h997-no?authuser=0" alt="mutmap_analysis_today" height="400px">

### Sliding window解析の概要

　実験で得られるデータは、精確に理論値の値のみが得られることはありません。少なからずランダムな要素が絡み、理論値からややはずれた値が得られることがほとんどです。

　MutMap解析でも、SNP-indexが0.5で得られると期待される場所で、0.48や0.60といった値が得られることがあります。ときには理論値（期待値）から外れた0.0や1.0が得られることもあります。しかし、その場所で多くの値を得て平均値を調べると、0.5に近い値が得られます。

　Sliding window解析は、任意の値（ここではSNP-index）の区間平均をゲノム全域にわたって調べて、区間平均（期待値）が極端に低くなる場所や高くなる場所を探す解析手法です。

<img src="https://lh3.googleusercontent.com/pw/ACtC-3euwLsOQQgwmqJWLWTeSROFjby5RYnNH_UdqTE9SEpefx_XkZILIKAdrOsq0bzT_7L6RZQFwAFgexYRUupN8c6EdZVUyNN96gSXsj6mTM8v26qaCZs2HCllVzuWexZZ2t8J_tAN80NbuloO79WWK5IR=w720-h405-no?authuser=0" alt="sliding_window_overview" height="300px">

　Sliding window解析は次の手順でおこないます。
1. 区間サイズとステップサイズ（次の区間への移動サイズ）、各区間の代表値を決める。  
<small>※ 代表値として「平均値」が使われることが多い。</small>
1. 1区間内に含まれるデータを取り出す。
1. 代表値を求める。
1. 区間をステップサイズ分だけ移動させる。
1. 手順2-4を繰り返し、全区間を調べる。

### サンプルデータのダウンロード





　[サンプルデータ](https://raw.githubusercontent.com/CropEvol/lecture/master/data/mutmap_chr10.txt)は、イネ第10染色体のSNP情報が書かれた **カンマ区切りファイル** です。

<img src="https://lh3.googleusercontent.com/pw/ACtC-3fNAxD422jnLtAXgJmheGNYjtxVoPwAAu3vGm27NfFEY035bmcdkESX2NlLBV-C3DmJNnwrrHprhXQex6B0GTNTMFDtHXEzMlqX-p1HyOE7atXWrof0DUpmgopj2-zSFBWf6v8dQDOmEZMEO2YU9dNc=w1147-h772-no?authuser=0" alt="mutmap_dataset" height="300px">

　次のコードセルを実行すると、そのカンマ区切りファイルが読み込まれ、pandasデータフレームの形に変換されます。また、各SNP座のSNP-indexも算出されて、`SNP_index`列として追加されます。

In [ ]:
"""
＊重要＊
最初にこのセルを実行してください。
この実習で使うサンプルファイルをダウンロードします。
"""
!wget -q -O mutmap_dataset.txt https://raw.githubusercontent.com/CropEvol/lecture/master/data/mutmap_chr10.txt

# pandasライブラリを準備する
import pandas as pd
# ファイルを読み込む
df = pd.read_csv("mutmap_dataset.txt", sep=',', header=0)
# SNP-indexを計算する
df["SNP_index"] = df["N_ALT"] / (df["N_REF"] + df["N_ALT"])
# データフレームの表示
df

## 今回の実習内容

1. 最初の1区間のSNP-indexの平均値を求める
  1. 最初の1区間のデータを取り出す
  1. 区間内の平均SNP-indexを求める
  1. 描画する
1. 2区間目、3区間目のSNP-indexの平均値も求める
  1. 区間内の平均SNP-indexを求める
  1. 描画する
1. 全区間にわたりSNP-indexの平均値を求める
  1. 各区間の平均SNP-indexの算出を自動化する
  1. 描画する

---

## 1. 最初の1区間のSNP-indexの平均を求める

このテキストでは、Sliding window解析の設定値は以下のとおりです。
- 区間サイズ: 1 Mb (= 1,000,000 bp)
- ステップサイズ: 0.2 Mb (= 200,000 bp)
- 区間内で取得するデータの代表値:
  - `POS`の平均値 <small>(※ 折れ線グラフを描く際にx軸座標の値になる)</small>
  - `SNP_index`の平均値 <small>(※ 折れ線グラフを描く際にy軸座標の値になる)</small>


### 1-1. 最初の1区間のデータを取り出す

　最初の1区間（`POS`が1-1,000,000の範囲）のデータを取り出してみましょう。

　下記のコードの`df[]`の中に、条件式を記述してください。

In [ ]:
# 最初の1区間のデータを取り出す
sub = df[]

# 取り出したデータの確認
sub

#### コード例

In [ ]:
# 最初の1区間のデータを取り出す
sub = df[ (df["POS"]>=1) & (df["POS"]<=1000000) ]

# 取り出したデータの確認
sub

### 1-2. 区間内の平均SNP-indexを求める

　次のコードセルには、区間内の`POS`の平均値を調べるコードをあらかじめ記述しています。そのコードを参考にして、区間内の`SNP_index`の平均値を調べるコードも記述しましょう。

```python
# 列データの平均値を調べる
列データ.mean()
# 例: 取り出した1区間のPOS列の平均値
sub["POS"].mean()
```



In [ ]:
# 区間内のPOSの平均値
p = sub["POS"].mean()

# 区間内のSNP_indexの平均値
s = 

print("mean POS: ", p)
print("mean SNP-index: ", s)

#### コード例

In [ ]:
# 区間内のPOSの平均値
p = sub["POS"].mean()

# 区間内のSNP_indexの平均値
s = sub["SNP_index"].mean()

print("mean POS: ", p)
print("mean SNP-index: ", s)

### 1-3. 描画する

　次のコードセルを追記せずに実行すると、全SNP-indexの散布図が描かれます。

　リスト`x_win`と`y_win`に情報を追記して、最初の区間平均のデータ（`POS`と`SNP_index`の平均値）が赤色の散布図で描かれるようにしましょう。

　<small>※ ここでは区間平均を散布図`plt.scatter()`で描きますが、のちほど折れ線グラフ`plt.plot()`に変更します。</small>

In [ ]:
# === グラフ用データ ===
# Sliding window代表値を格納するリスト
x_win = []
y_win = []

# 全SNP-indexのx軸座標、y軸座標
x = df["POS"]
y = df["SNP_index"]

# === グラフ描画 ===
import matplotlib.pyplot as plt
plt.figure(figsize=[12,4])    # グラフサイズの設定
plt.scatter(x, y)             # 散布図
plt.scatter(x_win, y_win, color="red")  # sliding windowの散布図
plt.title('SNP-index on chromosome 10', fontsize=18)  # グラフタイトル
plt.xlabel('Position (x 10 Mb)', fontsize=12)  # x軸ラベル
plt.ylabel('SNP-index', fontsize=12)           # y軸ラベル
plt.show() # グラフ表示

#### コード例

In [ ]:
# === グラフ用データ ===
# Sliding window代表値（x軸座標、y軸座標）を格納するリスト
x_win = [p]
y_win = [s]

# 全SNP-indexのx軸座標、y軸座標
x = df["POS"]
y = df["SNP_index"]

# === グラフ描画 ===
import matplotlib.pyplot as plt
plt.figure(figsize=[12,4])    # グラフサイズの設定
plt.scatter(x, y)             # 散布図
plt.scatter(x_win, y_win, color="red")  # sliding windowの散布図
plt.title('SNP-index on chromosome 10', fontsize=18)  # グラフタイトル
plt.xlabel('Position (x 10 Mb)', fontsize=12)  # x軸ラベル
plt.ylabel('SNP-index', fontsize=12)           # y軸ラベル
plt.show() # グラフ表示

## 2区間目、3区間目のSNP-indexの平均値も求める


### 2-1. 区間内の平均SNP-indexを求める

　ステップサイズが0.2 Mb (= 200,000 bp)の場合、2区間目と3区間目の`POS`の範囲は次のとおりです。
- 最初の区間(1区間目): 1 - 1,000,000 bp
- 2区間目: 200,001 - 1,200,000 bp
- 3区間目: 400,001 - 1,400,000 bp

　次のコードセルには、最初の区間（1区間目）の平均値を2つのリスト`x_win`と`y_win`に追加するコードを記述しています。

<small>※ 「最初の1区間のSNP-indexの平均を求める」でおこなったこととほぼ同じです。</small>

　そのコードを参考に、2区間目、３区間目についても同様に区間平均を調べて、リスト`x_win`と`y_win`にその値を追加してみてください。

```python
# リストへの追加 （Python基礎の復習です）
リスト.append(追加する値)
```

In [ ]:
# Sliding window代表値を格納するリスト
x_win = []
y_win = []

# 最初の区間（1区間目）の平均値を調べる
sub = df[ (df["POS"]>=1) & (df["POS"]<=1000000) ]
p = sub["POS"].mean()
s = sub["SNP_index"].mean()

x_win.append(p) # リストへの追加
y_win.append(s)

# 2区間目の平均値を調べる



# 3区間目の平均値を調べる



# リストの中身を確認
print(x_win)
print(y_win)

#### コード例

In [ ]:
# Sliding window代表値を格納するリスト
x_win = []
y_win = []

# 最初の区間（1区間目）の平均値を調べる
sub = df[ (df["POS"]>=1) & (df["POS"]<=1000000) ]
p = sub["POS"].mean()
s = sub["SNP_index"].mean()

x_win.append(p) # リストへの追加
y_win.append(s)

# 2区間目の平均値を調べる
sub = df[ (df["POS"]>=200001) & (df["POS"]<=1200000) ]
p = sub["POS"].mean()
s = sub["SNP_index"].mean()

x_win.append(p) # リストへの追加
y_win.append(s)


# 3区間目の平均値を調べる
sub = df[ (df["POS"]>=400001) & (df["POS"]<=1400000) ]
p = sub["POS"].mean()
s = sub["SNP_index"].mean()

x_win.append(p) # リストへの追加
y_win.append(s)


# リストの中身を確認
print(x_win)
print(y_win)

### 2-2. 描画する

　次のコードを実行すると、全SNP-indexの散布図と、最初の3区間の区間平均の散布図（赤色）が描かれます。

　次のコードセル5行目を変更して、区間平均が折れ線グラフで描かれるように変更してみましょう。

```python
# 散布図
plt.scatter(x軸座標データ, y軸座標データ, <オプション>)
# 折れ線グラフ
plt.plot(x軸座標データ, y軸座標データ, <オプション>)
```


In [ ]:
# === グラフ描画 ===
import matplotlib.pyplot as plt
plt.figure(figsize=[12,4])    # グラフサイズの設定
plt.scatter(x, y)             # 散布図
plt.scatter(x_win, y_win, color="red")  # sliding windowの散布図
plt.title('SNP-index on chromosome 10', fontsize=18)  # グラフタイトル
plt.xlabel('Position (x 10 Mb)', fontsize=12)  # x軸ラベル
plt.ylabel('SNP-index', fontsize=12)           # y軸ラベル
plt.show()

#### コード例

In [ ]:
# === グラフ描画 ===
import matplotlib.pyplot as plt
plt.figure(figsize=[12,4])    # グラフサイズの設定
plt.scatter(x, y)             # 散布図
#plt.scatter(x_win, y_win, color="red")  # sliding windowの散布図
plt.plot(x_win, y_win, color="red")  # sliding windowの折れ線グラフ
plt.title('SNP-index on chromosome 10', fontsize=18)  # グラフタイトル
plt.xlabel('Position (x 10 Mb)', fontsize=12)  # x軸ラベル
plt.ylabel('SNP-index', fontsize=12)           # y軸ラベル
plt.show()

## 3. 全区間にわたりSNP-indexの平均値を求める


### 3-1. 各区間の平均SNP-indexの算出を自動化する

　「2区間目、3区間目のSNP-indexの平均値も求める」では、区間平均を調べるコードを、それぞれの区間で数行を記述しました。同様の作業を、染色体の最後の区間までおこなうのはとても大変です。

　区間サイズやステップサイズは決まっており、最初の区間から最後の区間まで同様の作業を繰り返すだけですので、繰り返し構文を使えばその作業を自動化できるでしょう。

自動化させるコード
1. 取り出す区間の設定
1. 区間データの取り出し
1. 区間平均の算出
1. 区間平均を保存するリストへの追加  

　次のコードセルには、あらかじめそのコードを記述しています。詳細は省きますが、このコードの重要な点は以下のとおりです。
- 取り出す区間範囲（始点と終点）の初期値を設定する (12, 13行目のコード)
- 各ループ処理の間に、以下のことをおこなう。
  - 区間平均を調べて、データを一時保存する (17-23行目)
  - 次の区間のために区間範囲の始点と終点を更新する (26,27行目)
  - 終点が染色体サイズを超えたとき、ループ処理を終えるようにする （30,31行目）  
  <small>※ これを設定しないと、ループ処理が永遠に続く「無限ループ」と言われる状態になる。</small>

In [ ]:
# 染色体のサイズ（第10番染色体の全長 (bp)）
chrom_size = 23207287
# 区間サイズとステップサイズ
win_size  = 1   * 1000 * 1000  # 1Mb
step_size = 0.2 * 1000 * 1000  # 0.2Mb

# Sliding window代表値を格納するリスト
x_win = []
y_win = []

# 区間範囲の初期値
start = 1                    # 始点
end  = start + win_size - 1 # 終点

while True:
  # 区間データの取得
  sub = df[ (df["POS"]>=start) & (df["POS"]<=end) ]
  # 区間平均の取得
  p = sub["POS"].mean()
  s = sub["SNP_index"].mean()
  # リストへの追加
  x_win.append(p)
  y_win.append(s)

  # 区間の移動（区間範囲データにステップサイズを追加）
  start = start + step_size
  end  = end  + step_size

  # 終点が染色体サイズを超えたとき、ループを終える
  if end > chrom_size:
    break

# リストの中身を確認
print(x_win)
print(y_win)

### 3-2. 描画する

　「3-1. 各区間の平均SNP-indexの算出を自動化する」で得られた区間平均データを折れ線グラフにしてみましょう。

　下記のコードセルを実行してください。

In [ ]:
# === グラフ描画 ===
import matplotlib.pyplot as plt
plt.figure(figsize=[12,4])    # グラフサイズの設定
plt.scatter(x, y)             # 散布図
#plt.scatter(x_win, y_win, color="red")  # sliding windowの散布図
plt.plot(x_win, y_win, color="red")  # sliding windowの折れ線グラフ
plt.title('SNP-index on chromosome 10', fontsize=18)  # グラフタイトル
plt.xlabel('Position (x 10 Mb)', fontsize=12)  # x軸ラベル
plt.ylabel('SNP-index', fontsize=12)           # y軸ラベル
plt.show()

---
## まとめ

　このテキストでは、前回のMutMap解析にさらに手を加えて、SNP-indexの**Sliding window解析**をおこないました。

　SNP-indexの区間平均をゲノム全体にわたり調べてその推移を調べることにより、平均SNP-indexが高い領域（バルクDNAシーケンス中の突然変異の割合が高くなる領域）を図示しました。

　平均SNP-indexが高いゲノム領域には、葉の色を淡緑色（pale green）に変える遺伝子が存在するのでしょうか？ 

　イネ第10番染色体の22-23Mbの辺りには、「Chlorophyllide a oxygenase」をコードする遺伝子(*OsCAO1*)が存在しています。この遺伝子のノックアウト形質転換体（遺伝子の機能を壊した個体）は、葉の色が淡緑色になることが知られています(Abe et al., 2012)。






<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>